In [1]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import polars as pl
import random
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage,HumanMessage
from langchain.output_parsers import ResponseSchema,StructuredOutputParser
from typing import Dict,List,Tuple
import utils
import utils_model as um
%load_ext autoreload
%autoreload 2

In [2]:
model = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

C:\Users\monojit.mandal\AppData\Local\Temp\ipykernel_10152\2458642598.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  model = AzureChatOpenAI(


#### Extract Levers from Offer

In [3]:
offer_text = """
Price per unit: $100
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days (faster).
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.
"""
utils.extract_levers_from_text(
    offer_text = offer_text,
    model = model,
    example_data=pl.read_csv('data/sample_offers.csv')
)

Extracted levers:  {'price_per_unit': 100, 'quantity': 10000, 'bundling_unit': None, 'bundling_amount': None, 'bundling_discount': None, 'payment_term': 'NET10', 'payment_term_markup': 0, 'delivery_timeline': 7, 'contract_period': 1, 'contract_inflation': 0, 'rebates_threshold_unit': 11000, 'rebates_discount': 1, 'warranty': 1, 'incoterms': 'FOB'}


{'price_per_unit': 100,
 'quantity': 10000,
 'bundling_unit': None,
 'bundling_amount': None,
 'bundling_discount': None,
 'payment_term': 'NET10',
 'payment_term_markup': 0,
 'delivery_timeline': 7,
 'contract_period': 1,
 'contract_inflation': 0,
 'rebates_threshold_unit': 11000,
 'rebates_discount': 1,
 'warranty': 1,
 'incoterms': 'FOB'}

In [5]:
base_offer = '''
Price per unit: $100
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days.
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: FOB (Free on Board) - buyer is responsible for customs and delivery.
'''
MIN_LEVERS = {
    'Unit Price':100,
    'Quantity': 10000
}
MAX_LEVERS = {
    'Unit Price':120,
    'Quantity': 10000
}
levers = [
    'Unit Price','Bundling','Payment Term',
    'Delivery Timeline','Rebates','Warranties',
    'Incoterms'
]
supplier_offer = '''
Price per unit: $120
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days.
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: DDP.
'''
um.generate_offer_using_LLM(
    base_offer = base_offer,
    min_levers = MIN_LEVERS,
    max_levers = MAX_LEVERS,
    step = 3,
    TCO_hike_threshold_pct = 30,
    model = model,
    supplier_offer = supplier_offer,
    levers = ['Unit Price','Unit Price','Unit Price']
)
# utils.understand_priority_levers(
#     existing_offer=base_offer,
#     supplier_offer=supplier_offer,
#     model = model
# )['Levers']

TCO with existing offer:  1070178.08
Randomly selected levers are:  ['Unit Price']
Randomly selected levers are:  ['Unit Price']
Randomly selected levers are:  ['Unit Price']


{'offers': ['Unit Price: $ 111.5\nQuantity: 10,000 units\nBundling: No bundling option.\nPayment Terms: NET30 with a 3% markup if NET60 is requested.\nDelivery Timelines: 7 days.\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 1% rebate on orders above 11,000 units.\nWarranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.\nIncoterms: FOB (Free on Board) - buyer is responsible for customs and delivery.',
  'Unit Price: $ 109.50\nQuantity: 10,000 units\nBundling: No bundling option.\nPayment Terms: NET10, no option for extending to NET30.\nDelivery Timelines: 7 days.\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 1% rebate on orders above 11,000 units.\nWarranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.\nIncoterms: FOB (Free on Board) - buyer is responsible for customs and delivery.',
  'Unit Price: $ 117\nQuantity: 10

#### Generate equivalent offers using LLM

In [38]:
base_offer = """
Price per unit: $100
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days (faster).
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.
"""
supplier_offer = """
Price per unit: $200
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days (faster).
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: DDP.
"""
min_levers = {'Unit Price':0,'Quantity':10000}
max_levers = {'Unit Price':120,'Quantity':10000}
step=3
offers = um.generate_offer_using_LLM(
    base_offer = base_offer,
    min_levers = min_levers,
    max_levers = max_levers,
    step = step,
    TCO_hike_threshold_pct = 30,
    model = model,
    supplier_offer = supplier_offer,
    levers = [
        'Unit Price','Bundling','Payment Term',
        'Delivery Timeline','Rebates','Warranties',
        'Incoterms'
    ]
)
offers

TCO with existing offer:  1020178.08
{'Thoughts': 'The goal is to optimize the offer to ensure the annual cost of the contract is slightly less than $1,173,204.792 but greater than $1,020,178.08. I will focus on adjusting the unit price, warranty, and rebates while considering the cost implications of delivery timelines and incoterms. The safety stock and holding cost will also be considered since the delivery timeline impacts the safety stock level.', 'Offer': 'Unit Price: $ 97\nQuantity: 10,000 units\nBundling: No bundling option.\nPayment Terms: NET10, no option for extending to NET30.\nDelivery Timelines: 7 days (faster).\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 2% rebate on orders above 11,000 units.\nWarranties: 1-year standard warranty with an option to extend to 3 years for an additional $3/unit.\nIncoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.', 'Annual Cost of Contract ($)': 1124000.0}
{'Thoughts': 

{'offers': ['Unit Price: $ 97\nQuantity: 10,000 units\nBundling: No bundling option.\nPayment Terms: NET10, no option for extending to NET30.\nDelivery Timelines: 7 days (faster).\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 2% rebate on orders above 11,000 units.\nWarranties: 1-year standard warranty with an option to extend to 3 years for an additional $3/unit.\nIncoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.',
  'Unit Price: $ 100\nQuantity: 10,000 units\nBundling: No bundling option.\nPayment Terms: NET10\nDelivery Timelines: 7 days.\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 3% rebate on orders above 10,000 units.\nWarranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.\nIncoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.',
  'Unit Price: $100\nQuantity: 10,000 units\nRebates: 2% rebate if the 

In [23]:
from pprint import pprint
pprint(offers['offers'][2])

('Unit Price: $100\n'
 'Quantity: 10,000 units\n'
 'Bundling: If ordered with additional safety gear ($20,000 total), a 5% '
 'discount on the total contract.\n'
 'Payment Terms: NET30 with a 3% markup if NET60 is requested.\n'
 'Delivery Timelines: 7 days.\n'
 'Contract Period Length: 1 year with an option to renew at the same price.\n'
 'Rebates: 1% rebate if the purchase volume exceeds 11,000 units.\n'
 'Warranties: 1-year standard warranty with an option to extend to 3 years for '
 'an additional $5/unit.\n'
 'Incoterms: FOB (Free on Board) – buyer is responsible for customs and '
 'delivery.')


#### Offer Generator Agent

In [ ]:
# def generate_new_offer_using_LLM(
#     existing_offer: str,
#     TCO_min: float,
#     TCO_new: float,
#     levers: List[str],
#     model: AzureChatOpenAI
# ):
#     instruction_promt = """
#     Suppose you are an experienced Negotiator,a mathematical and logical expert for a Company. 
#     Your job is to generate an offers to Suppliers considering 
#     the annual cost of contract ($) same which will be instructed to you.

#     Your response will be in json format with keys:
#     - 'Thoughts': --keep your thoughts
#     - 'Offer': --keep your first offer text
#     - 'Annual Cost of Contract ($)' --annual cost intructed to you

#     A sample offer looks like this. Please note that this offer 
#     is just a sample offer. So the numbers are not exact here:
#     Unit Price: $ 95
#     Quantity: 10,000 units
#     Bundling: If ordered with additional safety gear ($20,000 total), a 5% discount on the total contract.
#     Payment Terms: NET30 with a 3% markup if NET60 is requested.
#     Delivery Timelines: 10 days.
#     Contract Period Length: 2 years with a 3% price increase in the second year.
#     Rebates: 2% rebate if the purchase volume exceeds 12,000 units in the first year.
#     Warranties: 1-year standard warranty.
#     Incoterms: DDP (Delivered Duty Paid, supplier manages all shipping, customs, and delivery logistics).

#     Also keep in mind of additional informations
#     1. When Payment term is NETxx it means
#     Supplier need payment within xx days, similarly when it payment term is NETyy it means 
#     Supplier need payment within yy days, considering rate of interest 6% when NETxx is chosen there
#     will be extra benefits since you will have (xx-yy) days to keep the money and generate 6% income. 
#     Please consider it in your calculation
#     2. There are extra cost implication when any incoterm is chosen. please check the cost details
#     for various incoterms:
#     EXW: $ 21000
#     FCA: $ 19000
#     FAS: $ 17000
#     FOB: $ 15000
#     CFR: $ 13000
#     CI: $ 11000
#     CPT: $ 9000
#     CIP: $ 7000
#     DAP: $ 5000
#     DPU: $ 3000
#     DDP: $0
#     3. If the delivery time is xx days then safety stock days is (1.5*xx) days.
#     Considering quantity as demand, Safety Stock Price = (quantity/365)*(price)*(safety stock days) $
#     Considering holding cost rate as 18%, Holding Cost = Safety Stock Price * 18/100 $
#     """
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#                 instruction_promt
#             ),
#             MessagesPlaceholder(variable_name="messages"),
#         ]
#     )

#     question = f"""
#     Consider the existing offer from Supplier
#     {existing_offer}
#     please generate your response in the required format given 
#     annual cost of contract $ {TCO_new}. It is absolutely okay 
#     even if annual cost of contract is slightly less than this. 
#     But the annual cost of contract should be greater tham $ {TCO_min}
#     You can change only these levers: {'.'.join(levers)}
#     """

#     response_schemas = [
#         ResponseSchema(name="Thoughts", description="Thought of LLM"),
#         ResponseSchema(name="Offer", description="Thought of LLM"),
#         ResponseSchema(name="Annual Cost of Contract ($)",description="Annual cost of contract"),
#     ]

#     output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

#     tco_calculator = prompt | model | output_parser
#     response = tco_calculator.invoke(
#         {
#             "messages": [HumanMessage(content=question)],
#         }
#     )
#     return response

In [ ]:
TCO_min = 1360000
TCO_new = 1400000
existing_offer = """
Price per unit: $100
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days (faster).
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.
"""
offers = utils.generate_eqv_offers_using_LLM(
    existing_offer = existing_offer,
    TCO_new = TCO_new,
    TCO_min = TCO_min,
    min_levers={
        'Quantity': 10000,
        'Unit Price': 90
    },
    max_levers={
        'Quantity': 20000,
        'Unit Price': 200
    },
    n_offers = 3,
    model = model
)
offers

Unit Price: $95
Quantity: 10,000 units
Bundling: If ordered with an additional service package (e.g., extended warranty or maintenance) at $15,000 total, a 5% discount on the total contract.
Payment Terms: NET10
Delivery Timelines: 7 days.
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate if the purchase volume exceeds 11,000 units.
Warranties: 1-year standard warranty.
Incoterms: DDP (Delivered Duty Paid, supplier manages all shipping, customs, and delivery logistics).
Unit Price: $100
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, with no option for extending to NET30.
Delivery Timelines: 10 days.
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: DDP (Delivered Duty Paid, supplier manages all shipping, customs, and delivery logis

['Unit Price: $95\nQuantity: 10,000 units\nBundling: If ordered with an additional service package (e.g., extended warranty or maintenance) at $15,000 total, a 5% discount on the total contract.\nPayment Terms: NET10\nDelivery Timelines: 7 days.\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 1% rebate if the purchase volume exceeds 11,000 units.\nWarranties: 1-year standard warranty.\nIncoterms: DDP (Delivered Duty Paid, supplier manages all shipping, customs, and delivery logistics).',
 'Unit Price: $100\nQuantity: 10,000 units\nBundling: No bundling option.\nPayment Terms: NET10, with no option for extending to NET30.\nDelivery Timelines: 10 days.\nContract Period Length: 1 year, with the option to renew at the same price.\nRebates: 1% rebate on orders above 11,000 units.\nWarranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.\nIncoterms: DDP (Delivered Duty Paid, supplier manages all shipping, custo

#### TCO calculator Agent

In [ ]:
TCO_promt = """
Suppose you are an experienced Negotiator,a mathematical and logical expert for a Company. 
Your job is to find out annual contract cost checking Supplier's offer input from
your company.

Your response will be in json format with keys: 'Thoughts','Annual Cost of Contract ($)'
'Thoughts' -- Keep your step by step thoughts
'Annual Cost of Contract ($)' -- Keep only numerical number of annual cost of contract

A sample Supplier's offer looks like this. Please note that this offer 
is just a sample offer. So the numbers are not exact here:
Price per unit: $ 95
Quantity: 10,000 units
Bundling: If ordered with additional safety gear ($20,000 total), a 5% discount on the total contract.
Payment Terms: NET30 with a 3% markup if NET60 is requested.
Delivery Timelines: 10 days.
Contract Period Length: 2 years with a 3% price increase in the second year.
Rebates: 2% rebate if the purchase volume exceeds 12,000 units in the first year.
Warranties: 1-year standard warranty.
Incoterms: DDP (Delivered Duty Paid, supplier manages all shipping, customs, and delivery logistics).

Also keep in mind of additional informations
1. When Payment term is NETxx it means
Supplier need payment within xx days, similarly when it payment term is NETyy it means 
Supplier need payment within yy days, considering rate of interest 6% when NETxx is chosen there
will be extra benefits since you will have (xx-yy) days to keep the money and generate 6% income. 
Please consider it in your calculation
2. There are extra cost implication when any incoterm is chosen. please check the cost details
for various incoterms:
EXW: $ 21000
FCA: $ 19000
FAS: $ 17000
FOB: $ 15000
CFR: $ 13000
CI: $ 11000
CPT: $ 9000
CIP: $ 7000
DAP: $ 5000
DPU: $ 3000
DDP: $0
3. If the delivery time is xx days then safety stock days is (1.5*xx) days.
Considering quantity as demand, Safety Stock Price = (quantity/365)*(price)*(safety stock days) $
Considering holding cost rate as 18%, Holding Cost = Safety Stock Price * 18/100 $
"""

In [14]:
base_offer = """
Price per unit: $100
Quantity: 10,000 units
Bundling: No bundling option.
Payment Terms: NET10, no option for extending to NET30.
Delivery Timelines: 7 days (faster).
Contract Period Length: 1 year, with the option to renew at the same price.
Rebates: 1% rebate on orders above 11,000 units.
Warranties: 1-year standard warranty with an option to extend to 3 years for an additional $5/unit.
Incoterms: FOB (Free on Board) – buyer is responsible for customs and delivery.
"""
proposal = """Quantity: 11,000 units"""
response = utils.calculate_TCO_from_offer_using_LLM(
    base_offer = base_offer,
    proposal = proposal,
    model = model
)
response

{'Thoughts': ['1. Calculate the base cost for 11,000 units at $100 per unit.',
  "2. No bundling or discount applies as per the supplier's offer.",
  '3. Payment terms are NET10, no extra financial benefit from NET30.',
  '4. Calculate the delivery cost for FOB, which is $15,000 as per the incoterms cost list.',
  '5. Determine the safety stock costs: Delivery time is 7 days, so safety stock days = 1.5 * 7 = 10.5 days.',
  '6. Calculate the Safety Stock Price: (11000 units / 365 days) * $100/unit * 10.5 days.',
  '7. Calculate the Holding Cost: Safety Stock Price * 18/100.',
  '8. Calculate the rebate: 1% rebate on orders above 11,000 units.',
  '9. There is no additional cost for extending the warranty as it is optional.',
  '10. Calculate the total annual cost by adding all relevant costs and subtracting any applicable rebates.'],
 'Annual Cost of Contract ($)': 1115573}

In [16]:
random.choices([1],[4],k=3)

[1, 1, 1]